In [1]:
import cv2
import os
import numpy as np
from rembg import remove
from pathlib import Path
from PIL import Image
import re      
import math
import scipy.spatial.distance
import imutils
from PIL import ImageFilter
import cvzone
import random

In [2]:
class_id_to_name = {
    0 : "plastic",
    1 : "paper",
    2 : "metal",
    3 : "glass",
    4 : "cardboard",
}

name_to_class_id = { v:k for k,v in class_id_to_name.items()}

In [3]:
"""
 This function receives a folder as input, resizes the images, for every image in this folder makes the background transparent, sharpens images,
 creates and saves 12 rotated images in a new folder. 6 of which comes from the rotation of the original image and 
 the other 6 from the rotation of the mirror image.
 
 """

def image_augmentation(folder):


    # Creates a new folder in the current directory
    new_path = 'c:/Users/vasil/Desktop/trashproject/{}_transparent_background'.format(folder)

    # 'mkdir' creates a directory in current directory.
    os.mkdir(new_path) 
    
    for filename in os.listdir(folder):

        image_path = os.path.abspath(filename) 
    

        # Replace the path
        input_path = f'{folder}\\'+image_path.split('\\')[-1].replace('\\', '/')
        
    
        # Reads the image
        input = cv2.imread(str(input_path))
        # Removes the background (makes image transparent)
        output = remove(input)

        # Resizes the output images in 400x300 pixels
        resized = cv2.resize(output, (400,300), interpolation = cv2.INTER_AREA)

        # Sharpens the image
        kernel = np.array([[0, -1, 0],
                            [-1, 5,-1],
                            [0, -1, 0]])
        image_sharp = cv2.filter2D(src=resized, ddepth=-1, kernel=kernel) # src= output

        # Mirror image
        num_rows, num_cols = image_sharp.shape[:2]

        src_points = np.float32([[0,0], [num_cols-1,0], [0,num_rows-1]])
        dst_points = np.float32([[num_cols-1,0], [0,0], [num_cols-1,num_rows-1]])
        matrix = cv2.getAffineTransform(src_points, dst_points)
        img_afftran = cv2.warpAffine(image_sharp, matrix, (num_cols,num_rows))

        img_list =[image_sharp,img_afftran]


        # Rotates the image
        a='sharp'
        for obj in img_list:
           
            for angle in range(0,360,60):
                img_rotate = imutils.rotate(obj,angle)
                output_path = Path(input_path).stem+f"_{a}_rotate{angle}.png"
            
                # Save the image
                cv2.imwrite(os.path.join(new_path,output_path), img_rotate)
            a = 'mirror'

In [12]:
"""This function takes the path of an image, reads the image, and finds the left, top ,right and bottom points of the image.
Then returns the Top-left point, the width and the height of the frame. """

def convertImage(image_path):
    image = Image.open(image_path) 
    img = image.convert("RGBA")
  
    datas = img.getdata()
  
    newData = datas
  
    L=(datas.size[0],0)
    R=(0,0)
    T=(0,0)
    B=(0,datas.size[1])
    for i in range(datas.size[0]):
        for j in range(datas.size[1]):
            item = datas.getpixel((i,j))
            if item[0] <15 and item[1] < 15 and item[2] < 15:
                newData.putpixel((i,j),(0,0,0,0))
            else:
                if (i<L[0]):
                    L=(i,j)
                if (i>R[0]):
                    R=(i,j)
                if (j>T[1]):
                    T=(i,j)
                if (j<B[1]):
                    B=(i,j)
                newData.putpixel((i,j),item)

    img.putdata(newData)

    pts = [L,R,T,B]
    X = (L[0],B[1])
    w = R[0]-L[0]
    h = T[1]-B[1]
    return X, w, h

In [5]:
""" This function takes a folder with images, removes their background 
and creates the vertical wave of them"""

def image_wave(folder):

    # Creates a new folder in the current directory
    new_path =  'c:/Users/vasil/Desktop/trashproject/{}_wave'.format(folder)

    # 'mkdir' creates a directory in current directory.
    os.mkdir(new_path) 
    
    for filename in os.listdir(folder):

        image_path = os.path.abspath(filename)
        # Replaces the path of the image 
        input_path = f'{folder}\\'+image_path.split('\\')[-1].replace('\\', '/')
        
    
        # Reads the image
        input = cv2.imread(str(input_path))
        # Removes the background
        output = remove(input)


        num_rows, num_cols = output.shape[:2]

        img_output = np.zeros(output.shape, dtype=output.dtype) 
        for i in range(num_rows): 
            for j in range(num_cols): 
                offset_x = int(25.0 * math.sin(2 * 3.14 * i / 180)) # the original was 1.75 and not 2
                offset_y = 0 
                if j+offset_x < num_rows: 
                    img_output[i,j] = output[i,(j+offset_x)%num_cols] 
                else: 
                    img_output[i,j] = 0 

        output_path = Path(input_path).stem+"_wave.png"

        # Saves image
        cv2.imwrite(os.path.join(new_path,output_path), img_output)

    

In [6]:
""" This function takes the destination points and the start points of an image"""

def find_coeffs(pa, pb):
    matrix = []
    for p1, p2 in zip(pa, pb):
        matrix.append([p1[0], p1[1], 1, 0, 0, 0, -p2[0]*p1[0], -p2[0]*p1[1]])
        matrix.append([0, 0, 0, p1[0], p1[1], 1, -p2[1]*p1[0], -p2[1]*p1[1]])

    A = np.matrix(matrix, dtype=np.float32)
    B = np.array(pb).reshape(8)

    res = np.dot(np.linalg.inv(A.T * A) * A.T, B)
    return np.array(res).reshape(8)

In [20]:
""" This function takes a folder with images, findes the left, right, bottom and top points of each image
    then uses the find_coeffs function in order to find the coefficients for the perspective transformation of each image. 
    Finally saves the images into a new folder """

def perpective_transformation(folder):

     # Creates a new folder in the current directory
    new_path = 'c:/Users/vasil/Desktop/trashproject/{}_perspective_transf'.format(folder)

    # 'mkdir' creates a directory in current directory.
    os.mkdir(new_path) 

    for filename in os.listdir(folder):

        image_path = os.path.abspath(filename)

        # Replace the path
        input_path = f'{folder}\\'+image_path.split('\\')[-1].replace('\\', '/') 
        img = Image.open(input_path)

        # Calls the convertImage function 
        X,w,h = convertImage(input_path)
        pts = np.float32((X,[X[0]+w,X[1]],[X[0]+w,X[1]+h],[X[0],X[1]+h]))

        TL = [0,0]
        TR = [0,0]

        TL[0] = X[0]+int(w/2)
        TL[1] = X[1]

        TR[0] = X[0]+w+int(w/2)
        TR[1] = X[1]

        # These are the destination points
        pa =np.float32((TL,TR, pts[2],pts[3]))
        
        width, height = img.size
    
                   
        # Calls the find_coeffs function
        coeffs = find_coeffs(pa, pts)

        # Makes the perspective transformation of an image
        image = img.transform((width, height), Image.PERSPECTIVE, coeffs)

        output_path = Path(input_path).stem+"_Perspective_Transform.png"

        # Saves image
        image = image.save(os.path.join(new_path,output_path))  
            
           

In [14]:
 """Creates a .txt file in the same directory as the image_path with a yolo format of the object frame

    :param image_path: path to the image
    :param X: top-left corner of the frame as (x,y)
    :param w: width of the frame
    :param h: height of the frame
    """

def create_yolo_txt_file(image_path, X, w, h):   
   
    img = Image.open(image_path)
    name = Path(image_path).stem

    # Grabs the first word of the path without the digits 
    name_alpha = ''.join([i for i in name.split('_')[0] if not i.isdigit()])

    # Replaces the variable name_alpha with its id acording to the dictionary name_to_class_id
    object_id = name_to_class_id.get(name_alpha)

    cX = (X[0]+w/2)/img.size[0]
    cY = (X[1]+h/2)/img.size[1]
    w = w/img.size[0]
    h = h/img.size[1]
    with open(str(Path(image_path).parent)+"/"+name+".txt", "w") as f:
        f.write(str(object_id) + " " + str(cX) + " " + str(cY) + " " + str(w) + " " + str(h))

In [8]:
""" This function resizes the backgound image in 800x600 pixels """

def resized_background(folder):
    # Creates a new folder in the current directory
    new_path =  'c:/Users/vasil/Desktop/trashproject/{}_resback'.format(folder)

    # 'mkdir' creates a directory in current directory.
    os.mkdir(new_path)

    for filename in os.listdir(folder):
        image_path = os.path.abspath(filename)

        #Replaces the path of the input image
        input_path = f'{folder}\\'+image_path.split('\\')[-1].replace('\\', '/')

        #Reads the image
        input = cv2.imread(str(input_path))

        # Resizes the input image
        resized = cv2.resize(input, (800,600), interpolation = cv2.INTER_AREA) 

        # Saves the image
        output_path = Path(input_path).stem+'.png' 
        cv2.imwrite(os.path.join(new_path,output_path), resized)

In [20]:
"""This function takes two folders,one with background images and one with transparent background objects,
for every image with transparent background choose a random background image to overlay it 
and saves them in a folder."""

def overlay__image(imgBack_folder,imgFront_folder,b):

    for filename in os.listdir(imgFront_folder):

        image_path = os.path.abspath(filename) 
       
        # Replace the path
        input_path_front = f'{imgFront_folder}\\'+image_path.split('\\')[-1].replace('\\', '/')
        
        # Calls the convertImage function
        X, w, h = convertImage(input_path_front)
        
        # Reads the image
        imgFront = cv2.imread(str(input_path_front), cv2.IMREAD_UNCHANGED)
        
        # Takes a random image from the imgBack_folder
        random_background = random.choice([x for x in os.listdir(imgBack_folder) 
                                            if os.path.isfile(os.path.join(imgBack_folder, x))]) 
        

        imageback_path = os.path.abspath(random_background)

        # Replace the path
        input_path_back = f'{imgBack_folder}\\'+imageback_path.split('\\')[-1].replace('\\', '/')

        # Reads the image
        imgBack = cv2.imread(str(input_path_back)) 

        # Overlays the two images in a specific place defined by b
        imgResult = cvzone.overlayPNG(imgBack,imgFront,b)

        #creates a new path for the imgResult
        imgResult_path = Path(input_path_front).stem+f"_background.png"
            
        # Save the image in the already existing folder result
        cv2.imwrite(os.path.join(os.path.abspath('testData'),imgResult_path), imgResult)

        # Creates a yolo txt file using a previous function
        create_yolo_txt_file('testData/'+imgResult_path,X,w,h)